# Project 2: Food Classifier


**Import libraries and Data**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


print(tf.__version__)

2.17.0


In [2]:
# Link zu kaggle.....geht das??

data_training = r"C:\Users\manu_\OneDrive - Universitaet Bern\03 HS24 UniBe-VIVO\05 Diabetes Management\archive\training"
data_evaluation = r"C:\Users\manu_\OneDrive - Universitaet Bern\03 HS24 UniBe-VIVO\05 Diabetes Management\archive\evaluation"
data_validation = r"C:\Users\manu_\OneDrive - Universitaet Bern\03 HS24 UniBe-VIVO\05 Diabetes Management\archive\validation"

**Load ResNet50 Model**


In [3]:
# Load Model
resNet50_model = ResNet50(weights='imagenet', include_top=False)

# Freeze all layers except the last 26
for layer in resNet50_model.layers[:-3]:   # adjust how many Layers you want to freeze
    layer.trainable = False


# Add a global average pooling layer to reduce the dimensionality
x = resNet50_model.output
x = GlobalAveragePooling2D()(x)         # without this there is a Dimensionality problem --> i dont understand this yet

# Add a fully connected output layer with the 11 food classes
output_layer = Dense(11, activation='softmax')(x)  # Use 'sigmoid' for binary classification, 'softmax' for multiclass

# Create the model with output layer
resNet50_model = Model(inputs=resNet50_model.input, outputs=output_layer)


# Compile the model
opt = Adam()
resNet50_model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall()])


# Display the model architecture
# resNet50_model.summary()

**Prepare Training Data**

In [5]:
# TRAINING

#set the image height and width to 240 and 320 respectively
img_height, img_width = 240, 320

#set a batchsize (you can use 8/16)
bs = 32

# Perform data augmentation using Image Data Generator; apply rescaling, rotation of 30, shearing of 0.15 and zoom of 0.15
train_datagen = ImageDataGenerator(
    dtype='float32',
    rescale=1./255,                              # rescale pixel values to range 0 to 1
    #rotation_range=30,                          # randomly rotates image up to 30°  -> useless?
    #shear_range=0.15,                           # randomly sheers image
    #zoom_range=0.15,                            # random zoom
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    #horizontal_flip=True,                       
    featurewise_center=True,                    # normalization
    featurewise_std_normalization=True,         # normalization
        )

# mean RGB specific
# mean will be subtracted from all pixels -> all values around zero -> makes training faster
# how to get the specific numbers???
train_datagen.mean = np.array([93/255, 112/255, 128/255], dtype=np.float32).reshape((1,1,3))
# image divided by standard deviation -> standardized spread of pixel values
train_datagen.std = np.array([73/255, 70/255, 72/255], dtype=np.float32).reshape((1,1,3))



# Preprocess training data
training_set = train_datagen.flow_from_directory(
    data_training,                                      # Directory path containing images
    target_size=(img_height, img_width),                # resize images
    batch_size = bs,                                    # number of images being processed at once
    class_mode='categorical')                           # changed from binary to categorical???

Found 9866 images belonging to 11 classes.


**?????Prepare Validation Data ------------ Or Evaluation?????**

In [6]:
evaluation_datagen = ImageDataGenerator(          # no rescaling, rotation, shearing and zoom
    dtype='float32',
    rescale=1./255, 
    featurewise_center=True,
    featurewise_std_normalization=True)

evaluation_datagen.mean = np.array([93/255, 112/255, 128/255], dtype=np.float32).reshape((1,1,3))    # why these numbers??
evaluation_datagen.std = np.array([73/255, 70/255, 72/255], dtype=np.float32).reshape((1,1,3))

evaluation_set = evaluation_datagen.flow_from_directory(
    data_evaluation,
    target_size=(img_height, img_width),
    batch_size=bs,
    class_mode='categorical')

Found 3347 images belonging to 11 classes.


**Train and Evaluate ResNet50**

In [14]:
epoch_no = 5                
steps=len(training_set)     # steps -> number of batches in training set

print(steps, "Batches of images need to be processed in one Epoch.")

309 Batches of images need to be processed in one Epoch.


In [8]:
h=resNet50_model.fit(
    training_set,
    validation_data=evaluation_set,             
    steps_per_epoch=steps,                                          # defines how many batches should be processed in one epoch
    epochs=epoch_no,                                                # one epoch => model has seen every data in training set once
    callbacks=[EarlyStopping(monitor='val_loss',patience=5)])

c:\Users\manu_\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
 16/309 ━━━━━━━━━━━━━━━━━━━━ 6:56 1s/step - accuracy: 0.0727 - loss: 0.4278 - precision: 0.0000e+00 - recall: 0.0000e+00

KeyboardInterrupt: 